<a href="https://colab.research.google.com/github/EdWangLoDaSc/DSVOT/blob/main/NOAA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pykrige

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.interpolate import griddata
from PIL import Image
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import (
    Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D, Cropping2D,
    AveragePooling2D, Flatten, Reshape, Dropout, Conv2D, LSTM,
    RepeatVector
)
from tensorflow.keras.losses import MSE
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from skimage.metrics import structural_similarity as ssim
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

import h5py
from tqdm import tqdm


from pykrige.ok3d import OrdinaryKriging3D
from pykrige.uk3d import UniversalKriging3D

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 真实无预处理

In [ ]:
import numpy as np
from tqdm import tqdm
from scipy.interpolate import griddata

def generate_data(f, sen_num_kind_list, sen_num_var_list):
    lat = np.array(f['lat'])
    lon = np.array(f['lon'])
    sst = np.array(f['sst'])
    sst1 = np.nan_to_num(sst)
    sst_reshape = sst[0, :].reshape(len(lat[0, :]), len(lon[0, :]), order='F')
    xv1, yv1 = np.meshgrid(lon[0, :], lat[0, :])

    X_ki = np.zeros((1040 * len(sen_num_kind_list) * len(sen_num_var_list), len(lat[0, :]), len(lon[0, :]), 2))
    y_ki = np.zeros((1040 * len(sen_num_kind_list) * len(sen_num_var_list), len(lat[0, :]), len(lon[0, :]), 1))

    for ki in range(len(sen_num_kind_list)):
        sen_num = sen_num_kind_list[ki]
        X_va = np.zeros((1040 * len(sen_num_var_list), len(lat[0, :]), len(lon[0, :]), 2))
        y_va = np.zeros((1040 * len(sen_num_var_list), len(lat[0, :]), len(lon[0, :]), 1))

        for va in range(len(sen_num_var_list)):
            X_t = np.zeros((1040, len(lat[0, :]), len(lon[0, :]), 2))
            y_t = np.zeros((1040, len(lat[0, :]), len(lon[0, :]), 1))

            for t in range(1040):
                y_t[t, :, :, 0] = np.nan_to_num(sst[t, :].reshape(len(lat[0, :]), len(lon[0, :]), order='F'))

                np.random.seed(sen_num_var_list[va])
                sparse_locations_lat = np.random.randint(len(lat[0, :]), size=(sen_num))
                sparse_locations_lon = np.random.randint(len(lon[0, :]), size=(sen_num))

                sparse_locations = np.column_stack((sparse_locations_lat, sparse_locations_lon))

                for s in range(sen_num):
                  a, b = sparse_locations[s]
                  while np.isnan(sst_reshape[int(a), int(b)]) == True:
                    a = np.random.randint(len(lat[0, :]), size=(1))
                    b = np.random.randint(len(lon[0, :]), size=(1))
                    sparse_locations[s, 0] = a
                    sparse_locations[s, 1] = b


                sparse_data = np.zeros((sen_num))
                for s in range(sen_num):
                    sparse_data[s] = (y_t[t, :, :, 0][int(sparse_locations[s, 0]), int(sparse_locations[s, 1])])

                sparse_locations_ex = np.zeros(sparse_locations.shape)
                for i in range(sen_num):
                    sparse_locations_ex[i, 0] = lat[0, :][int(sparse_locations[i, 0])]
                    sparse_locations_ex[i, 1] = lon[0, :][int(sparse_locations[i, 1])]

                grid_z0 = griddata(sparse_locations_ex, sparse_data, (yv1, xv1), method='nearest')
                for j in range(len(lon[0, :])):
                    for i in range(len(lat[0, :])):
                        if np.isnan(sst_reshape[i, j]):
                            grid_z0[i, j] = 0
                X_t[t, :, :, 0] = grid_z0
                mask_img = np.zeros(grid_z0.shape)
                for i in range(sen_num):
                    mask_img[int(sparse_locations[i, 0]), int(sparse_locations[i, 1])] = 1
                X_t[t, :, :, 1] = mask_img

            X_va[1040 * va:1040 * (va + 1), :, :, :] = X_t
            y_va[1040 * va:1040 * (va + 1), :, :, :] = y_t

        X_ki[(1040 * len(sen_num_var_list)) * ki:(1040 * len(sen_num_var_list)) * (ki + 1), :, :, :] = X_va
        y_ki[(1040 * len(sen_num_var_list)) * ki:(1040 * len(sen_num_var_list)) * (ki + 1), :, :, :] = y_va

    return X_ki, y_ki


f = h5py.File('/content/drive/MyDrive/Physics/Physics/sst_weekly.mat', 'r')
sen_num_kind_list = [200,240,280,320]
sen_num_var_list = [300, 100,10]

X_ki, y_ki = generate_data(f, sen_num_kind_list, sen_num_var_list)


In [ ]:
np.save('/content/drive/MyDrive/Physics/Dataset10/x_NOAA_train.npy',X_ki)
np.save('/content/drive/MyDrive/Physics/Dataset10/y_NOAA_train.npy',y_ki)


In [ ]:
sen_num_kind_list = [200,240,280,300,320,340]
sen_num_var_list = [900]

X_ki, y_ki = generate_data(f, sen_num_kind_list, sen_num_var_list)

In [ ]:
np.save('/content/drive/MyDrive/Physics/Dataset10/x_NOAA_test.npy',X_ki)
np.save('/content/drive/MyDrive/Physics/Dataset10/y_NOAA_test.npy',y_ki)

# VOR-CNN

In [ ]:
x_train = np.load('/content/drive/MyDrive/Physics/Dataset10/x_NOAA_train.npy',mmap_mode = 'r')
y_train = np.load('/content/drive/MyDrive/Physics/Dataset10/y_NOAA_train.npy',mmap_mode = 'r')
x_test = np.load('/content/drive/MyDrive/Physics/Dataset10/x_NOAA_test.npy',mmap_mode = 'r')
y_test = np.load('/content/drive/MyDrive/Physics/Dataset10/y_NOAA_test.npy',mmap_mode = 'r')

In [ ]:
f = h5py.File('/content/drive/MyDrive/Physics/Physics/sst_weekly.mat', 'r')
lat = np.array(f['lat'])
lon = np.array(f['lon'])
sst = np.array(f['sst'])
time = np.array(f['time'])

sst1= np.nan_to_num(sst)

In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.layers import Input, Add, Dense, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D, Flatten, Reshape, LSTM
from keras.models import Model
from keras import backend as K
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from scipy.spatial import Voronoi
import math
from scipy.interpolate import griddata


import tensorflow as tf

input_img = Input(shape=(len(lat[0,:]),len(lon[0,:]),2))
x = Conv2D(48, (7,7),activation='relu', padding='same')(input_img)
x = Conv2D(48, (7,7),activation='relu', padding='same')(x)
x = Conv2D(48, (7,7),activation='relu', padding='same')(x)
x = Conv2D(48, (7,7),activation='relu', padding='same')(x)
x = Conv2D(48, (7,7),activation='relu', padding='same')(x)
x = Conv2D(48, (7,7),activation='relu', padding='same')(x)
x = Conv2D(48, (7,7),activation='relu', padding='same')(x)
x_final = Conv2D(1, (7,7), padding='same')(x)
model = Model(input_img, x_final)
model.compile(optimizer='adam', loss='mse')

model_cb=ModelCheckpoint('/content/drive/MyDrive/Physics/NOAA/Model_NOAA.h5', monitor='val_loss',save_best_only=True,verbose=1)
early_cb=EarlyStopping(monitor='val_loss', patience=50,verbose=1)
cb = [model_cb, early_cb]
history = model.fit(x_train,y_train,epochs=150,batch_size=32,verbose=1,callbacks=cb,shuffle=True,validation_data=[x_test, y_test])

In [ ]:
from tensorflow.keras.models import load_model
model_cb=ModelCheckpoint('/content/drive/MyDrive/Physics/NOAA/Model_NOAA.h5', monitor='val_loss',save_best_only=True,verbose=1)
early_cb=EarlyStopping(monitor='val_loss', patience=50,verbose=1)
cb = [model_cb, early_cb]
# Load the entire model from the .h5 file
model = load_model('/content/drive/MyDrive/Physics/NOAA/Model_NOAA.h5')
history = model.fit(x_train,y_train,epochs=150,batch_size=32,verbose=1,callbacks=cb,shuffle=True,validation_data=[x_test, y_test])

## Load Pre-trained Vor-CNN

In [ ]:
vorcnn = models.load_model('/content/drive/MyDrive/Physics/NOAA/Model_NOAA.h5')

x_train = np.load('/content/drive/MyDrive/Physics/Dataset10/x_NOAA_train.npy',mmap_mode = 'r')
y_train = np.load('/content/drive/MyDrive/Physics/Dataset10/y_NOAA_train.npy',mmap_mode = 'r')
x_test = np.load('/content/drive/MyDrive/Physics/Dataset10/x_NOAA_test.npy',mmap_mode = 'r')
y_test = np.load('/content/drive/MyDrive/Physics/Dataset10/y_NOAA_test.npy',mmap_mode = 'r')

In [ ]:
import time

start_time = time.time()

# 进行预测
res_test_cnn = vorcnn.predict(x_test)

# 记录预测结束时间
end_time = time.time()

# 计算并打印预测所需的时间
inference_time = end_time - start_time
print(f"Inference time: {inference_time} seconds")

In [ ]:
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

def calculate_metrics(original_images, predicted_images):
    ssim_scores = []
    psnr_scores = []
    l2_norms = []

    for original, predicted in zip(original_images, predicted_images):
        # Calculate SSIM
        ssim_score = ssim(original, predicted, multichannel=True)
        ssim_scores.append(ssim_score)

        # Calculate PSNR
        psnr_score = psnr(original, predicted, data_range=predicted.max() - predicted.min())
        psnr_scores.append(psnr_score)

        # Calculate L2 norm
        l2_norm = np.linalg.norm(original - predicted)
        l2_norms.append(l2_norm)

    return ssim_scores, psnr_scores, l2_norms

# Initialize lists to store metrics for each subdataset
all_ssim_scores = []
all_psnr_scores = []
all_l2_norms = []

# Calculate and print metrics for different subdatasets
for i in range(len(y_test) // 1040):
    y_subset = y_test[1040*i:1040*(i+1)]
    res_subset = res_test_cnn[1040*i:1040*(i+1)]

    # Calculate SSIM, PSNR, and L2 norm
    ssim_scores, psnr_scores, l2_norms = calculate_metrics(y_subset, res_subset)

    # Print average SSIM, PSNR, and L2 norm for the current subset
    print(f"Subset {i+1}:")
    print(f"Average SSIM: {sum(ssim_scores) / len(ssim_scores)}")
    print(f"Average PSNR: {sum(psnr_scores) / len(psnr_scores)}")
    print(f"Average L2 Norm: {sum(l2_norms) / len(l2_norms)}")
    print()


In [ ]:
plt.imshow(res_test_cnn[3])

In [ ]:
np.save('vorcnn.npy',res_test_cnn[3])

# CED-LSTM

In [ ]:
x_train = np.load('/content/drive/MyDrive/Physics/Dataset10/x_NOAA_train.npy',mmap_mode = 'r')
y_train = np.load('/content/drive/MyDrive/Physics/Dataset10/y_NOAA_train.npy',mmap_mode = 'r')
x_test = np.load('/content/drive/MyDrive/Physics/Dataset10/x_NOAA_test.npy',mmap_mode = 'r')
y_test = np.load('/content/drive/MyDrive/Physics/Dataset10/y_NOAA_test.npy',mmap_mode = 'r')

x_train = np.load('/content/drive/MyDrive/Physics/Dataset10/x_NOAA_train.npy',mmap_mode = 'r')
y_train = np.load('/content/drive/MyDrive/Physics/Dataset10/y_NOAA_train.npy',mmap_mode = 'r')
x_test = np.load('/content/drive/MyDrive/Physics/Dataset10/x_NOAA_test.npy',mmap_mode = 'r')
y_test = np.load('/content/drive/MyDrive/Physics/Dataset10/y_NOAA_test.npy',mmap_mode = 'r')


'''x_data = np.vstack([x_train,x_test])

y_data = np.vstack([y_train,y_test])'''


x_data = np.load('/content/drive/MyDrive/Physics/Dataset10/x_NOAA.npy',mmap_mode = 'r')
y_data = np.load('/content/drive/MyDrive/Physics/Dataset10/y_NOAA.npy',mmap_mode = 'r')


In [ ]:
np.save('/content/drive/MyDrive/Physics/Dataset10/x_NOAA.npy',x_data)
np.save('/content/drive/MyDrive/Physics/Dataset10/y_NOAA.npy',y_data)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

tf.keras.utils.set_random_seed(1)


# Encoder
input_img = layers.Input(shape=(180, 360, 1))
x = layers.Conv2D(32, (7, 7), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(16, (7, 7), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)  # Added to match dimensions correctly
x = layers.Flatten()(x)
encoded = layers.Dense(512, activation='relu', name='encoded')(x)

# Decoder
x = layers.Dense(45*90*16, activation='relu')(encoded)  # Adjusted to match the encoder output
x = layers.Reshape((45, 90, 16))(x)
x = layers.Conv2D(16, (7, 7), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(32, (7, 7), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (7, 7), activation='linear', padding='same', name='decoded')(x)

# Autoencoder Model
autoencoder = models.Model(input_img, decoded)

autoencoder.compile(optimizer='adam', loss='mae')
autoencoder.summary()


In [ ]:
x_train = x_train[:,:,:,0]
x_test = x_test[:,:,:,0]
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# Set the random seed for reproducibility
tf.keras.utils.set_random_seed(12)


# Callback to save the best model
model_checkpoint = ModelCheckpoint(
    filepath='/content/drive/MyDrive/Physics/NOAA/best_model_CED.h5',  # 保存模型的路径
    monitor='val_loss',  # 监视的指标
    save_best_only=True,  # 仅保存在验证集上性能最好的模型
    verbose=1,  # 日志等级
    mode='min',  # 监视指标的目标是最小化
    save_format='h5'  # 模型保存格式
)


# Callbacks for adaptive learning rate, early stopping, and model checkpointing
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    patience=40,
    min_lr=1e-5
)
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=60,
    restore_best_weights=True
)

# Training the autoencoder
history = autoencoder.fit(
    x_train,
    y_train,
    epochs=150,
    batch_size=16,
    callbacks=[reduce_lr, early_stop, model_checkpoint],  # Include model_checkpoint in the callbacks
    validation_data=(x_test, y_test),
    shuffle=True
)


In [ ]:
# Assume you have already trained the autoencoder model

# Get the encoder part
encoder_input = autoencoder.input
encoder_output = autoencoder.get_layer('encoded').output
encoder = models.Model(encoder_input, encoder_output)

# Get the decoder part
decoder_input = layers.Input(shape=(512,))
decoder_layers = autoencoder.layers[7:]  # Assuming the encoded layer is the 7th layer
decoder_output = decoder_layers[0](decoder_input)  # Pass the input to the first layer of the decoder
for layer in decoder_layers[1:]:
    decoder_output = layer(decoder_output)  # Pass the output through each subsequent layer
decoder = models.Model(decoder_input, decoder_output)


## Load Pre-trained

In [ ]:
x_train = np.load('/content/drive/MyDrive/Physics/Dataset10/x_NOAA_train.npy',mmap_mode = 'r')
y_train = np.load('/content/drive/MyDrive/Physics/Dataset10/y_NOAA_train.npy',mmap_mode = 'r')
x_test = np.load('/content/drive/MyDrive/Physics/Dataset10/x_NOAA_test.npy',mmap_mode = 'r')
y_test = np.load('/content/drive/MyDrive/Physics/Dataset10/y_NOAA_test.npy',mmap_mode = 'r')

x_data = np.load('/content/drive/MyDrive/Physics/Dataset10/x_NOAA.npy',mmap_mode = 'r')
y_data = np.load('/content/drive/MyDrive/Physics/Dataset10/y_NOAA.npy',mmap_mode = 'r')



In [ ]:
np.save('/content/drive/MyDrive/Physics/Dataset10/x_NOAA.npy',x_data)
np.save('/content/drive/MyDrive/Physics/Dataset10/y_NOAA.npy',y_data)

In [ ]:
autoencoder = models.load_model('/content/drive/MyDrive/Physics/NOAA/best_model_CED.h5')

encode_layers = autoencoder.layers[:7]

# Create a new Sequential model with the extracted layers
encoded = tf.keras.models.Sequential(encode_layers)


last_layers = autoencoder.layers[-7:]
new_model = tf.keras.models.Sequential(last_layers)
#encoded_features = encoded.predict(x_data[:,:,:,0])
reconstructed_images = autoencoder.predict(x_test[:,:,:,0])

In [ ]:
# Assuming encoded_features is obtained from the encoder_model
encoded_features = encoder_model.predict(x_test[:,:,:,0])

# Decode the encoded features using the custom DecoderLayer
decoded_images_custom = decoder_layer(encoded_features[:10])

# Reconstruct images directly from the autoencoder
reconstructed_images_autoencoder = autoencoder.predict(x_test[:,:,:,0])



In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, decoder_model, **kwargs):
        super(DecoderLayer, self).__init__(**kwargs)
        # Ensure the decoder_model is a callable (e.g., a Keras model or a function)
        if not callable(decoder_model):
            raise ValueError("decoder_model must be callable")
        self.decoder_model = decoder_model

    def call(self, inputs):
        # It's good practice to check if the inputs are valid for the decoder_model
        # This can be more specific based on the expected input shape, type, etc.
        if inputs is None:
            raise ValueError("Input to DecoderLayer cannot be None")

        # Use decoder_model to decode the inputs
        decoded_images = self.decoder_model(inputs)
        return decoded_images

## Plot NOAA CED

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl

# 调整全局字体大小和颜色条标签大小
mpl.rcParams['font.size'] = 8
mpl.rcParams['axes.labelsize'] = 6  # 提高了labelsize以提升可读性

def plot_image_and_colorbar(ax, image, title, mask=None, cmap='viridis', cbar_labelsize=6, vmin=None, vmax=None):
    # 如果提供了mask，将mask位置的值设置为NaN
    if mask is not None:
        image = np.where(mask, np.nan, image)
    im = ax.imshow(image, cmap=cmap, vmin=vmin, vmax=vmax)  # 使用vmin和vmax
    ax.set_title(title, fontsize=11)  # 增大标题字体
    ax.axis('off')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = plt.colorbar(im, cax=cax)
    cbar.ax.tick_params(labelsize=10)  # 减小刻度字体

def plot_and_save_comparison_multi(true_images, vor_images, reconstructed_images, indices, dpi=300):
    ncols = len(indices)
    nrows = 4

    # 确定颜色标尺范围
    vmin = np.nanmin(true_images)
    vmax = np.nanmax(true_images)
    error_vmax = np.nanmax(np.abs(true_images - reconstructed_images))-25

    fig, axs = plt.subplots(nrows, ncols, figsize=(3 * ncols, 6), dpi=dpi)

    for col, index in enumerate(indices):
        mask = true_images[index, :, :, 0] == 0

        # 真实图像
        plot_image_and_colorbar(axs[0, col], true_images[index, :, :, 0], "(a) NOAA State Field", mask=mask, vmin=vmin, vmax=vmax)

        # 重建图像
        plot_image_and_colorbar(axs[1, col], reconstructed_images[index, :, :, 0], "(b) NOAA Reconstructed Fields", mask=mask, vmin=vmin, vmax=vmax)

        # 误差图
        error_map = np.abs(true_images[index, :, :, 0] - reconstructed_images[index, :, :, 0])
        plot_image_and_colorbar(axs[2, col], error_map, "(c) Error Map", mask=mask, cmap='viridis', vmin=0, vmax=error_vmax)

        # Voronoi图像
        plot_image_and_colorbar(axs[3, col], vor_images[index, :, :, 0], "(d) NOAA Voronoi Fields", vmin=vmin, vmax=vmax)

    plt.tight_layout()
    plt.savefig('CED-NOAA.png', dpi=dpi, bbox_inches='tight')
    plt.show()

# 使用示例
plot_and_save_comparison_multi(y_test, x_test, reconstructed_images, indices=[80, 200, 400, 800], dpi=500)

In [ ]:
np.max(reconstructed_images[1])

In [ ]:
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import numpy as np

def apply_mask_and_calculate_ssim_psnr(y_true, y_pred, mask):
    # 在 y_pred 中应用 mask，mask 为 True 的地方设置为 0
    y_pred_masked = np.where(mask, 0, y_pred)

    # 直接计算 SSIM 和 PSNR
    ssim_val = ssim(y_true, y_pred_masked, data_range=y_true.max() - y_true.min())
    psnr_val = psnr(y_true, y_pred_masked, data_range=y_true.max() - y_true.min())

    return ssim_val, psnr_val

# 假设 y_test 和 reconstructed_images 已经是 np.array 形式
ssim_values = []
psnr_values = []

# 这里假设 mask 的形状与 y_test 和 reconstructed_images 中的每张图像相同
mask = y_test == 0  # 假设 mask 是根据 y_test 中的 0 值创建的

for i in range(y_test.shape[0]):
    ssim_val, psnr_val = apply_mask_and_calculate_ssim_psnr(y_test[i, :, :, 0], reconstructed_images[i, :, :, 0], mask[i, :, :, 0])
    ssim_values.append(ssim_val)
    psnr_values.append(psnr_val)

# 计算平均 SSIM 和 PSNR
mean_ssim = np.mean(ssim_values)
mean_psnr = np.mean(psnr_values)

print(f"Average SSIM: {mean_ssim}")
print(f"Average PSNR: {mean_psnr}")


## LSTM Data

In [ ]:
encoded_features = encoded.predict(x_data[:,:,:,0])
datasets = np.split(encoded_features, 18)

# 随机选择数据集用作训练和测试
train_indices, test_indices = train_test_split(np.arange(18), test_size=1/3, shuffle = False)#random_state=42

# 准备训练和测试数据
def prepare_data(indices, datasets, input_length=3, forecast_horizon=3):
    X, y = [], []
    for idx in indices:
        data = datasets[idx]

        for i in range(len(data) - input_length - forecast_horizon + 1):
            X.append(data[i:(i + input_length)])
            y.append(data[(i + input_length):(i + input_length + forecast_horizon)])
    return np.array(X), np.array(y)
x_train, y_train = prepare_data(train_indices, datasets)
np.save('/content/drive/MyDrive/Physics/Dataset10/x_NOAA_LSTM_train.npy',x_train)
np.save('/content/drive/MyDrive/Physics/Dataset10/y_NOAA_LSTM_train.npy',y_train)

del x_train, y_train

x_val, y_val = prepare_data(test_indices, datasets)

np.save('/content/drive/MyDrive/Physics/Dataset10/x_NOAA_LSTM_val.npy',x_val)
np.save('/content/drive/MyDrive/Physics/Dataset10/y_NOAA_LSTM_val.npy',y_val)

del x_val, y_val


In [ ]:

x_NOAA_LSTM_train = np.load('/content/drive/MyDrive/Physics/Dataset10/x_NOAA_LSTM_train.npy',mmap_mode = 'r')
y_NOAA_LSTM_train = np.load('/content/drive/MyDrive/Physics/Dataset10/y_NOAA_LSTM_train.npy',mmap_mode = 'r')
x_NOAA_LSTM_val = np.load('/content/drive/MyDrive/Physics/Dataset10/x_NOAA_LSTM_val.npy',mmap_mode = 'r')
y_NOAA_LSTM_val = np.load('/content/drive/MyDrive/Physics/Dataset10/y_NOAA_LSTM_val.npy',mmap_mode = 'r')

In [ ]:
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import tensorflow as tf

# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=40,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

model_checkpoint = ModelCheckpoint(
    filepath='/content/drive/MyDrive/Physics/NOAA/best_model_ori_NOAA_3steps_mae.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=1,
    mode='min'
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,        # Reduce learning rate by 20%
    patience=30,       # Number of epochs with no improvement after which learning rate will be reduced
    verbose=1,
    mode='min',
    min_delta=0.0001,  # Minimum change to qualify as an improvement
    cooldown=0,        # Number of epochs to wait before resuming normal operation after lr has been reduced
    min_lr=0           # Lower bound on the learning rate
)

def build_optimized_model(n_step=3, feature_dim=512):
    """
    Builds an optimized LSTM model with added regularization and adjusted complexity.

    Parameters:
    - n_step (int): The number of time steps per sequence.
    - feature_dim (int): The dimensionality of the input features.

    Returns:
    - model (tf.keras.Model): The constructed LSTM-based model with optimizations.
    """
    inputs = Input(shape=(n_step, feature_dim))
    x = layers.LSTM(128, return_sequences=True)(inputs)  # Reduce LSTM units to balance model complexity
    x = layers.LSTM(128)(x)
    x = layers.RepeatVector(n_step)(x)
    x = layers.TimeDistributed(layers.Dense(feature_dim))(x)
    outputs = layers.Activation('relu')(x)

    model = Model(inputs=inputs, outputs=outputs, name="OptimizedModel")
    return model

# Instantiate and compile the optimized model
optimized_model = build_optimized_model()
optimized_model.compile(
    loss='mae',
    optimizer=tf.keras.optimizers.Adam(0.001)  # Adjusted learning rate
)

optimized_model.summary()

best_val_loss = float('inf')  # Initialize best validation loss
best_seed = None  # Initialize best seed

# Loop through different random seeds
for seed in range(5):  # Example: trying seeds from 0 to 9
    tf.keras.backend.clear_session()  # Clear previous model to reset randomness
    tf.random.set_seed(seed)  # Set random seed

    # Rebuild and compile the model with the current seed
    optimized_model = build_optimized_model()
    optimized_model.compile(loss='mae', optimizer=tf.keras.optimizers.Adam(0.001))

    # Train the model
    history = optimized_model.fit(
        x_NOAA_LSTM_train, y_NOAA_LSTM_train,
        validation_data=(x_NOAA_LSTM_val, y_NOAA_LSTM_val),
        epochs=150,
        batch_size=16,
        shuffle=True,
        callbacks=[early_stopping, model_checkpoint, reduce_lr]
    )

    # Check if the current seed resulted in a better validation loss
    if min(history.history['val_loss']) < best_val_loss:
        best_val_loss = min(history.history['val_loss'])
        best_seed = seed

print(f"Best seed: {best_seed}, Best validation loss: {best_val_loss}")





## Load Pre-trained LSTM

In [ ]:

x_NOAA_LSTM_train = np.load('/content/drive/MyDrive/Physics/Dataset10/x_NOAA_LSTM_train.npy',mmap_mode = 'r')
y_NOAA_LSTM_train = np.load('/content/drive/MyDrive/Physics/Dataset10/y_NOAA_LSTM_train.npy',mmap_mode = 'r')
x_NOAA_LSTM_val = np.load('/content/drive/MyDrive/Physics/Dataset10/x_NOAA_LSTM_val.npy',mmap_mode = 'r')
y_NOAA_LSTM_val = np.load('/content/drive/MyDrive/Physics/Dataset10/y_NOAA_LSTM_val.npy',mmap_mode = 'r')

In [ ]:
lstm = models.load_model('/content/drive/MyDrive/Physics/NOAA/best_model_ori_NOAA_3steps_mae.h5')


In [ ]:

y_test_splitted = np.split(y_test, 6)

y_test_cropped = [images[3:-2,:,:, :] for images in y_test_splitted]

y_test_final = np.array(y_test_cropped)

print("Final shape of y_test:", y_test_final.shape)



In [ ]:
restruct_images_lstm.shape

In [ ]:
import matplotlib.pyplot as plt

# 可视化重建图像
fig, axs = plt.subplots(3, 2, figsize=(10, 15))  # 创建子图布局

# 计算 Colorbar 的范围
vmin = y_test[0].min()
vmax = y_test[0].max()

for i in range(3):
    # 绘制原始图像
    im1 = axs[i, 0].imshow(reconstructed_images[i, :, :, 0], vmin=vmin, vmax=vmax)
    axs[i, 0].set_title('Original Image {}'.format(i+1))  # 设置标题
    axs[i, 0].axis('off')  # 关闭坐标轴

    # 绘制LSTM重建图像
    im2 = axs[i, 1].imshow(restruct_images_lstm[i, :, :, 0], vmin=vmin, vmax=vmax)
    axs[i, 1].set_title('Reconstructed Image LSTM {}'.format(i+1))  # 设置标题
    axs[i, 1].axis('off')  # 关闭坐标轴

# 创建 Colorbar
cbar1 = fig.colorbar(im1, ax=axs[:, 0])
cbar2 = fig.colorbar(im2, ax=axs[:, 1])

plt.show()



In [ ]:
res_test_lst = lstm.predict(x_NOAA_LSTM_val)[0,:,:]
decoder_layer = DecoderLayer(new_model)

restruct_images_lstm = np.array(decoder_layer(res_test_lst[:1035,:]))
restruct_images_true = np.array(decoder_layer(x_NOAA_LSTM_val[:,0,:][:10]))

restruct_images_lstm.shape

In [ ]:
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

def calculate_metrics(original_images, predicted_images):
    ssim_scores = []
    psnr_scores = []
    l2_norms = []

    for original, predicted in zip(original_images, predicted_images):
        # Calculate SSIM
        ssim_score = ssim(original, predicted, multichannel=True)
        ssim_scores.append(ssim_score)

        # Calculate PSNR
        psnr_score = psnr(original, predicted, data_range=predicted.max() - predicted.min())
        psnr_scores.append(psnr_score)

        # Calculate L2 norm
        l2_norm = np.linalg.norm(original - predicted)
        l2_norms.append(l2_norm)

    return ssim_scores, psnr_scores, l2_norms

# Calculate SSIM, PSNR, and L2 norm
ssim_scores, psnr_scores, l2_norms = calculate_metrics(y_test_final[0,:,:,:,:], restruct_images_lstm)

# Print average SSIM, PSNR, and L2 norm
print(f"Average SSIM: {sum(ssim_scores) / len(ssim_scores)}")
print(f"Average PSNR: {sum(psnr_scores) / len(psnr_scores)}")
print(f"Average L2 Norm: {sum(l2_norms) / len(l2_norms)}")


In [ ]:
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

def apply_mask(original_images, predicted_images):
    masked_predicted_images = []
    for original, predicted in zip(original_images, predicted_images):
        # 创建一个掩码，其中原始图像的像素值为0的位置为True
        mask = original == 0

        # 使用掩码更新预测图像，将对应于掩码为True的位置的像素值设置为0
        masked_predicted = np.where(mask, 0, predicted)
        masked_predicted_images.append(masked_predicted)

    return masked_predicted_images

def calculate_metrics(original_images, predicted_images):
    ssim_scores = []
    psnr_scores = []
    l2_norms = []

    for original, predicted in zip(original_images, predicted_images):
        # Calculate SSIM
        ssim_score = ssim(original, predicted, multichannel=True)
        ssim_scores.append(ssim_score)

        # Calculate PSNR
        psnr_score = psnr(original, predicted, data_range=predicted.max() - predicted.min())
        psnr_scores.append(psnr_score)

        # Calculate L2 norm
        l2_norm = np.linalg.norm(original - predicted)
        l2_norms.append(l2_norm)

    return ssim_scores, psnr_scores, l2_norms

# 假设 y_test_final 是原始图像数组，restruct_images_lstm 是预测图像数组
# 首先，将原始图像中的0值部分作为掩码应用到预测图像上
masked_predicted_images = apply_mask(y_test_final[0,:,:,:,:], restruct_images_lstm)

# 然后，计算更新后的预测图像与原始图像之间的SSIM、PSNR和L2 norm
ssim_scores, psnr_scores, l2_norms = calculate_metrics(y_test_final[0,:,:,:,:], masked_predicted_images)

# 打印平均SSIM、PSNR和L2 norm
print(f"Average SSIM: {np.mean(ssim_scores)}")
print(f"Average PSNR: {np.mean(psnr_scores)}")
print(f"Average L2 Norm: {np.mean(l2_norms)}")


In [ ]:
np.min(restruct_images_lstm[100])

In [ ]:
def calculate_metrics_subset(original_images, predicted_images):
    ssim_scores = []
    psnr_scores = []
    l2_norms = []

    for original, predicted in zip(original_images, predicted_images):
        # 创建一个掩码，其中原始图像的像素值为0的位置为True
        mask = original == 0

        # 使用掩码更新预测图像，将对应于掩码为True的位置的像素值设置为0
        masked_predicted = np.where(mask, 0, predicted)

        # Calculate SSIM
        ssim_score = ssim(original, masked_predicted, multichannel=True)
        ssim_scores.append(ssim_score)

        # Calculate PSNR
        psnr_score = psnr(original, masked_predicted, data_range=masked_predicted.max() - masked_predicted.min())
        psnr_scores.append(psnr_score)

        # Calculate L2 norm
        l2_norm = np.linalg.norm(original - masked_predicted)
        l2_norms.append(l2_norm)

    return ssim_scores, psnr_scores, l2_norms
import time

# Initialize lists to store metrics for each subset
all_ssim_scores = []
all_psnr_scores = []
all_l2_norms = []
all_inference_times = []

# Iterate over subsets of images and calculate metrics
for i in range(len(y_test_final)):
    # Calculate metrics for the current subset
    start_time = time.time()
    restruct_images_lstm = np.array(decoder_layer(res_test_lst[1035*i:1035*(i+1),:]))
    res_test_lst = lstm.predict(x_NOAA_LSTM_val)[:,0,:]

    end_time = time.time()

    # Calculate inference time
    inference_time = end_time - start_time
    all_inference_times.append(inference_time)

    # Calculate SSIM, PSNR, and L2 norms
    ssim_scores, psnr_scores, l2_norms = calculate_metrics_subset(y_test_final[i], restruct_images_lstm)

    # Append metrics to the lists
    all_ssim_scores.append(ssim_scores)
    all_psnr_scores.append(psnr_scores)
    all_l2_norms.append(l2_norms)

# Compute the average metrics for each subset
avg_ssim_scores = [sum(scores) / len(scores) for scores in all_ssim_scores]
avg_psnr_scores = [sum(scores) / len(scores) for scores in all_psnr_scores]
avg_l2_norms = [sum(norms) / len(norms) for norms in all_l2_norms]
avg_inference_time = sum(all_inference_times) / len(all_inference_times)

# Print the average metrics for each subset
for i in range(len(avg_ssim_scores)):
    print(f"Subset {i+1}:")
    print(f"Average SSIM: {avg_ssim_scores[i]}")
    print(f"Average PSNR: {avg_psnr_scores[i]}")
    print(f"Average L2 Norm: {avg_l2_norms[i]}")
    print(f"Inference Time: {avg_inference_time} seconds")


In [ ]:
import matplotlib.pyplot as plt

# Define the data
methods = ['CED-LSTM', 'Vor-CNN', 'ConvLSTM']  # Add ConvLSTM
levels = [200, 240, 280, 300, 320, 340]
ssim_ced_lstm = [0.8358804598332382, 0.835554236306358, 0.8490705987148428, 0.8525863788543473, 0.8454216991970055, 0.8443145120122972]  # Update with new data
psnr_ced_lstm = [36.33874952578177, 35.59074896236341, 36.802145647367006, 37.13562049675428, 36.490809481591576, 36.826683817498804]  # Update with new data
ssim_vor_cnn = [0.579636193, 0.598724861, 0.603436399, 0.61198152, 0.611980767, 0.61406588]
psnr_vor_cnn = [29.09964257, 30.49640416, 30.62988655, 30.99013098, 30.95251542, 31.52378384]
ssim_conv_lstm = [0.6215, 0.6545, 0.648, 0.6636, 0.6548, 0.6612]  # Add SSIM data for ConvLSTM
psnr_conv_lstm = [27.8333, 29.0194, 29.1142, 30.2744, 29.1651, 30.4262]  # Add PSNR data for ConvLSTM

# Create the plot
plt.figure(figsize=(12, 4))

# Plot SSIM values
plt.subplot(1, 2, 1)
for method, ssim_scores, color, marker in zip(methods, [ssim_ced_lstm, ssim_vor_cnn, ssim_conv_lstm], ['blue', 'orange', 'green'], ['o', '^', 's']):
    plt.plot(levels, ssim_scores, marker=marker, linestyle='-', color=color, label=method)
plt.title('SSIM Comparison', fontsize=16)
plt.xlabel('Level', fontsize=14)
plt.ylabel('SSIM', fontsize=14)
plt.xticks(levels)
plt.grid(True)
plt.legend(loc='lower right', fontsize='small')  # Decrease legend size

# Plot PSNR values
plt.subplot(1, 2, 2)
for method, psnr_scores, color, marker in zip(methods, [psnr_ced_lstm, psnr_vor_cnn, psnr_conv_lstm], ['blue', 'orange', 'green'], ['o', '^', 's']):
    plt.plot(levels, psnr_scores, marker=marker, linestyle='-', color=color, label=method)
plt.title('PSNR Comparison', fontsize=16)
plt.xlabel('Level', fontsize=14)
plt.ylabel('PSNR', fontsize=14)
plt.xticks(levels)
plt.grid(True)
plt.legend(loc='lower right', fontsize='small')  # Decrease legend size

plt.tight_layout()

plt.savefig('cnn-CEDLSTM-ConvLSTM-compare.png', dpi=300)

plt.show()


In [ ]:
start_time = time.time()
res_test_lst = lstm.predict(x_NOAA_LSTM_val)[:,0,:]

end_time = time.time()
# Calculate inference time
inference_time = end_time - start_time
inference_time

In [ ]:
np.save('ced_lstm_4.npy',restruct_images_true[0])

In [ ]:
plt.imshow(restruct_images_true[3])

# ConvLSTM

## Data Normlization

In [ ]:
x_train = np.load('/content/drive/MyDrive/Physics/Dataset10/x_NOAA_train.npy',mmap_mode = 'r')[:,:,:,0]
y_train = np.load('/content/drive/MyDrive/Physics/Dataset10/y_NOAA_train.npy',mmap_mode = 'r')
x_test = np.load('/content/drive/MyDrive/Physics/Dataset10/x_NOAA_test.npy',mmap_mode = 'r')[:,:,:,0]
y_test = np.load('/content/drive/MyDrive/Physics/Dataset10/y_NOAA_test.npy',mmap_mode = 'r')

In [ ]:
def scale_data(data):
    # Initialize the scaled data array with the same shape as the input data
    scaled_data = np.zeros_like(data, dtype=np.float32)
    # Initialize arrays to store the min and max values for each sample
    min_vals = np.zeros((data.shape[0], 1, 1, 1))
    max_vals = np.zeros_like(min_vals)

    # Iterate through each sample in the data
    for i in range(data.shape[0]):
        # Calculate the min and max values for the current sample
        min_vals[i] = np.min(data[i])
        max_vals[i] = np.max(data[i])
        # Apply the scaling operation
        scaled_data[i] = (data[i] - min_vals[i]) / (max_vals[i] - min_vals[i])

    return scaled_data, min_vals, max_vals

x_train_scaled,_,_ = scale_data(x_train)
x_test_scaled,_,_ = scale_data(x_test)
y_train_scaled,_,_ = scale_data(y_train)
y_test_scaled,min_vals,max_vals = scale_data(y_test)


In [ ]:
y_test_scaled,min_vals,max_vals = scale_data(y_test)


In [ ]:
max_vals[:20,:,:,:]

In [ ]:

np.save('/content/drive/MyDrive/Physics/Dataset10/x_train_scaled.npy', x_train_scaled)
np.save('/content/drive/MyDrive/Physics/Dataset10/x_test_scaled.npy', x_test_scaled)
np.save('/content/drive/MyDrive/Physics/Dataset10/y_train_scaled.npy', y_train_scaled)
np.save('/content/drive/MyDrive/Physics/Dataset10/y_test_scaled.npy', y_test_scaled)
# 保存 min_vals 和 max_vals
np.save('/content/drive/MyDrive/Physics/Dataset10/y_test_min_vals.npy', min_vals)
np.save('/content/drive/MyDrive/Physics/Dataset10/y_test_max_vals.npy', max_vals)


In [ ]:
# 加载数据
x_train_scaled = np.load('/content/drive/MyDrive/Physics/Dataset10/x_train_scaled.npy', mmap_mode='r+')
x_test_scaled = np.load('/content/drive/MyDrive/Physics/Dataset10/x_test_scaled.npy', mmap_mode='r+')
y_train_scaled = np.load('/content/drive/MyDrive/Physics/Dataset10/y_train_scaled.npy', mmap_mode='r+')
y_test_scaled = np.load('/content/drive/MyDrive/Physics/Dataset10/y_test_scaled.npy', mmap_mode='r+')
x_train_scaled = x_train_scaled.reshape(x_train_scaled.shape[0], 180, 360, 1)
x_test_scaled = x_test_scaled.reshape(x_test_scaled.shape[0], 180, 360, 1)


# 加载 min_vals 和 max_vals
min_vals = np.load('/content/drive/MyDrive/Physics/Dataset10/y_test_min_vals.npy')
max_vals = np.load('/content/drive/MyDrive/Physics/Dataset10/y_test_max_vals.npy')


In [ ]:
import numpy as np

def prepare_convLSTM_data(x_data, y_data, num_of_sources, input_length=3, forecast_horizon=3):
    X, y = [], []

    # 计算每个源的样本数量
    samples_per_source = len(x_data) // num_of_sources

    for i in range(num_of_sources):
        # 根据每个源的样本数量计算起始和结束索引
        start_idx = i * samples_per_source
        end_idx = start_idx + samples_per_source

        # 切分x_data和y_data，获取当前源的数据
        x_source_data = x_data[start_idx:end_idx]
        y_source_data = y_data[start_idx:end_idx]

        # 将每个样本的源数量合并成一个新的维度
        x_sample = x_source_data.reshape(samples_per_source, 180,360,1)
        y_sample = y_source_data.reshape(samples_per_source, 180,360,1)

        # 将每个样本划分为输入和输出序列
        for j in range(samples_per_source - input_length - forecast_horizon + 1):
            X.append(x_sample[j:j+input_length])
            y.append(y_sample[j+input_length:j+input_length+forecast_horizon])

    return np.array(X), np.array(y)

# 定义输入和输出长度
input_length = 3
forecast_horizon = 3

# 计算源的数量
num_of_sources = len(x_train_scaled) // 1040

# 准备ConvLSTM训练数据集
'''x_convLSTM_train, y_convLSTM_train = prepare_convLSTM_data(x_train_scaled, y_train_scaled, num_of_sources, input_length, forecast_horizon)
np.save('/content/drive/MyDrive/Physics/Dataset10/x_convLSTM_train.npy', x_convLSTM_train)
np.save('/content/drive/MyDrive/Physics/Dataset10/y_convLSTM_train.npy', y_convLSTM_train)
print("x_convLSTM_train shape:", x_convLSTM_train.shape)
print("y_convLSTM_train shape:", y_convLSTM_train.shape)
del x_convLSTM_train,y_convLSTM_train'''
# 准备ConvLSTM测试数据集
x_convLSTM_test, y_convLSTM_test = prepare_convLSTM_data(x_test_scaled, y_test_scaled, num_of_sources, input_length, forecast_horizon)
np.save('/content/drive/MyDrive/Physics/Dataset10/x_convLSTM_test.npy', x_convLSTM_test)
np.save('/content/drive/MyDrive/Physics/Dataset10/y_convLSTM_test.npy', y_convLSTM_test)
print("x_convLSTM_test shape:", x_convLSTM_test.shape)
print("y_convLSTM_test shape:", y_convLSTM_test.shape)



In [ ]:
import numpy as np

def prepare_convLSTM_data(min_vals, max_vals, num_of_sources, input_length=3, forecast_horizon=3):
    X, y = [], []

    # 计算每个源的样本数量
    samples_per_source = len(min_vals) // num_of_sources

    for i in range(num_of_sources):
        # 根据每个源的样本数量计算起始和结束索引
        start_idx = i * samples_per_source
        end_idx = start_idx + samples_per_source

        # 切分x_data和y_data，获取当前源的数据
        x_source_data = min_vals[start_idx:end_idx]
        y_source_data = max_vals[start_idx:end_idx]

        # 将每个样本的源数量合并成一个新的维度
        x_sample = x_source_data.reshape(samples_per_source, 1,1,1)
        y_sample = y_source_data.reshape(samples_per_source, 1,1,1)

        # 将每个样本划分为输入和输出序列
        for j in range(samples_per_source - input_length - forecast_horizon + 1):
            X.append(x_sample[j+input_length:j+input_length+forecast_horizon])
            y.append(y_sample[j+input_length:j+input_length+forecast_horizon])

    return np.array(X), np.array(y)

# 定义输入和输出长度
input_length = 3
forecast_horizon = 3

# 计算源的数量
num_of_sources = len(x_train_scaled) // 1040
min_vals, max_vals
# 准备ConvLSTM训练数据集
min_vals, max_vals = prepare_convLSTM_data(min_vals, max_vals, num_of_sources, input_length, forecast_horizon)
np.save('/content/drive/MyDrive/Physics/Dataset10/min_vals_convlstm.npy', min_vals)
np.save('/content/drive/MyDrive/Physics/Dataset10/max_vals_convlstm.npy', max_vals)
print("x_convLSTM_train shape:", min_vals.shape)
print("y_convLSTM_train shape:", max_vals.shape)


## Load Data

In [ ]:
import numpy as np

# Load all scaled data
x_convLSTM_train_scaled = np.load('/content/drive/MyDrive/Physics/Dataset10/x_convLSTM_train.npy', mmap_mode='r')
y_convLSTM_train_scaled = np.load('/content/drive/MyDrive/Physics/Dataset10/y_convLSTM_train.npy', mmap_mode='r')
x_convLSTM_test_scaled = np.load('/content/drive/MyDrive/Physics/Dataset10/x_convLSTM_test.npy', mmap_mode='r')
y_convLSTM_test_scaled = np.load('/content/drive/MyDrive/Physics/Dataset10/y_convLSTM_test.npy', mmap_mode='r')
min_vals_convlstm = np.load('/content/drive/MyDrive/Physics/Dataset10/min_vals_convlstm.npy', mmap_mode='r')
max_vals_convlstm = np.load('/content/drive/MyDrive/Physics/Dataset10/max_vals_convlstm.npy', mmap_mode='r')

y_convLSTM_test_restored = np.load('/content/drive/MyDrive/Physics/Dataset10/y_convLSTM_test_restored.npy', mmap_mode='r')

# Access the loaded data
print("x_convLSTM_train_scaled shape:", x_convLSTM_train_scaled.shape)
print("y_convLSTM_train_scaled shape:", y_convLSTM_train_scaled.shape)
print("x_convLSTM_test_scaled shape:", x_convLSTM_test_scaled.shape)
print("y_convLSTM_test_scaled shape:", y_convLSTM_test_scaled.shape)


In [ ]:
def restore_original_scale(scaled_data, min_vals, max_vals):
    # Ensure min_vals and max_vals are broadcastable to the shape of scaled_data
    # This might require reshaping min_vals and max_vals depending on how they were saved
    restored_data = scaled_data * (max_vals - min_vals) + min_vals
    return restored_data

In [ ]:
np.save('/content/drive/MyDrive/Physics/Dataset10/y_convLSTM_test_restored.npy', y_convLSTM_test_restored)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define input shape
input_shape = (None, 3, 180, 360, 1)

# Input layer
inputs = layers.Input(shape=input_shape[1:])

# ConvLSTM layers
x = layers.ConvLSTM2D(
    filters=16,
    kernel_size=(7, 7),
    padding="same",
    return_sequences=True,
    activation="relu",
)(inputs)

x = layers.ConvLSTM2D(
    filters=16,
    kernel_size=(7, 7),
    padding="same",
    return_sequences=True,
    activation="relu",
)(x)

x = layers.ConvLSTM2D(
    filters=16,
    kernel_size=(7, 7),
    padding="same",
    return_sequences=True,
    activation="relu",
)(x)

# Output layer
outputs = layers.Conv3D(
    filters=1,
    kernel_size=(7, 7, 7),
    activation="linear",
    padding="same"
)(x)

# Create the model
model = models.Model(inputs, outputs)

# Compile the model
model.compile(loss=tf.keras.losses.mean_absolute_error, optimizer=tf.keras.optimizers.Adam())

# Print model summary
model.summary()


In [ ]:
from tensorflow.keras import backend as K
K.clear_session()


In [ ]:

from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

'''# Set the random seed for reproducibility
tf.keras.utils.set_random_seed(12)'''
#tf.keras.mixed_precision.set_global_policy('mixed_float16')

model = models.load_model('/content/drive/MyDrive/Physics/NOAA/best_model_convlstm_retrain.h5')

# Callback to save the best model
model_checkpoint = ModelCheckpoint(
    filepath='/content/drive/MyDrive/Physics/NOAA/best_model_convlstm_retrain.h5',  # 保存模型的路径
    monitor='val_loss',  # 监视的指标
    save_best_only=True,  # 仅保存在验证集上性能最好的模型
    verbose=1,  # 日志等级
    mode='min',  # 监视指标的目标是最小化
    save_format='h5'  # 模型保存格式
)


# Callbacks for adaptive learning rate, early stopping, and model checkpointing
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    patience=30,
    min_lr=1e-5
)
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=45,
    restore_best_weights=True
)

# Training the autoencoder
history = model.fit(
    x_convLSTM_train_scaled,
    y_convLSTM_train_scaled,
    epochs=10,
    batch_size=4,
    callbacks=[reduce_lr, model_checkpoint],  # Include model_checkpoint in the callbacks
    validation_data=(x_convLSTM_test_scaled, y_convLSTM_test_scaled),
    shuffle=True
)


In [ ]:
model.save('/content/drive/MyDrive/Physics/NOAA/best_model_convlstm_retrain.h5')

In [ ]:
# Training the autoencoder
history = model.fit(
    x_convLSTM_train_scaled,
    y_convLSTM_train_scaled,
    epochs=50,
    batch_size=8,
    callbacks=[reduce_lr, model_checkpoint],  # Include model_checkpoint in the callbacks
    validation_data=(x_convLSTM_test_scaled, y_convLSTM_test_scaled),
    shuffle=True
)

In [ ]:
model = models.load_model('/content/drive/MyDrive/Physics/NOAA/best_model_convlstm_final.h5')
model_checkpoint = ModelCheckpoint(
    filepath='/content/drive/MyDrive/Physics/NOAA/best_model_convlstm_final.h5',  # 保存模型的路径
    monitor='val_loss',  # 监视的指标
    save_best_only=True,  # 仅保存在验证集上性能最好的模型
    verbose=1,  # 日志等级
    mode='min',  # 监视指标的目标是最小化
    save_format='h5'  # 模型保存格式
)


# Callbacks for adaptive learning rate, early stopping, and model checkpointing
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    patience=30,
    min_lr=1e-5
)
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=45,
    restore_best_weights=True
)

# Training the autoencoder
history = model.fit(
    x_convLSTM_train_scaled,
    y_convLSTM_train_scaled,
    epochs=10,
    batch_size=4,
    callbacks=[reduce_lr, model_checkpoint],  # Include model_checkpoint in the callbacks
    validation_data=(x_convLSTM_test_scaled, y_convLSTM_test_scaled),
    shuffle=True
)

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# Enable mixed precision training
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Set the random seed for reproducibility
tf.random.set_seed(12)

# Callback to save the best model
model_checkpoint = ModelCheckpoint(
    filepath='/content/drive/MyDrive/Physics/NOAA/best_model_convlstm.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=1,
    mode='min'
)

# Callbacks for adaptive learning rate, early stopping, and model checkpointing
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    patience=40,
    min_lr=1e-5
)
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=60,
    restore_best_weights=True
)

# Build and compile your model
model.compile(loss='mean_squared_error', optimizer='adam')

# Training the model
history = model.fit(
    x_convLSTM_train,
    y_convLSTM_train,
    epochs=150,
    batch_size=4,
    callbacks=[reduce_lr, early_stop, model_checkpoint],
    validation_data=(x_convLSTM_test, y_convLSTM_test),
    shuffle=True
)



## Test Performance

In [ ]:
x_convLSTM_train_scaled.shape

In [ ]:
model = models.load_model('/content/drive/MyDrive/Physics/NOAA/best_model_convlstm_32units.h5')

res_lstm = model.predict(x_convLSTM_test_scaled)

res_lstm_restored = restore_original_scale(res_lstm, min_vals_convlstm, max_vals_convlstm)


In [ ]:
import matplotlib.pyplot as plt

# Data
num_sensors = [200, 240, 280, 300, 320, 340]
ssim = [0.6215, 0.6545, 0.648, 0.6636, 0.6548, 0.6612]
psnr = [27.8333, 29.02, 29.1142, 30.2744, 29.1651, 30.4262]

# Plotting SSIM
plt.figure(figsize=(10, 3))  # 调整图的宽度
plt.plot(num_sensors, ssim, color='blue', marker='o', label='SSIM')
plt.xlabel('Number of Sensors', fontsize=14)
plt.ylabel('SSIM', fontsize=14)
plt.title('SSIM vs. Number of Sensors')
plt.grid(True)

# Plotting PSNR
plt.figure(figsize=(10, 3))  # 调整图的宽度
plt.plot(num_sensors, psnr, color='orange', marker='o', label='PSNR')
plt.xlabel('Number of Sensors', fontsize=14)
plt.ylabel('PSNR', fontsize=14)
plt.title('PSNR vs. Number of Sensors')
plt.grid(True)

plt.show()



In [ ]:
import matplotlib.pyplot as plt

# Data
num_sensors = [200, 240, 280, 300, 320, 340]
ssim = [0.6215, 0.6545, 0.648, 0.6636, 0.6548, 0.6612]
psnr = [27.8333, 29.02, 29.1142, 30.2744, 29.1651, 30.4262]

# Plotting SSIM and PSNR together
fig, ax1 = plt.subplots(figsize=(10, 5))  # 调整图的大小
ax1.plot(num_sensors, ssim, color='blue', marker='o', label='SSIM')
ax1.set_xlabel('Number of Sensors', fontsize=14)  # 调整横坐标标签的字体大小
ax1.set_ylabel('SSIM', color='blue', fontsize=14)  # 调整纵坐标标签的字体大小
ax1.tick_params(axis='y', labelcolor='blue')
ax1.tick_params(axis='both', labelsize=12)  # 调整横纵坐标数字的字体大小
ax1.grid(True)

ax2 = ax1.twinx()  # 创建第二个坐标轴
ax2.plot(num_sensors, psnr, color='orange', marker='o', label='PSNR')
ax2.set_ylabel('PSNR', color='orange', fontsize=14)  # 调整纵坐标标签的字体大小
ax2.tick_params(axis='y', labelcolor='orange')
ax2.tick_params(axis='both', labelsize=12)  # 调整横纵坐标数字的字体大小

plt.title('SSIM and PSNR vs. Number of Sensors', fontsize=16)  # 调整标题字体大小

fig.tight_layout()  # 调整布局以防止重叠
plt.show()



In [ ]:
thf = res_lstm[-1030:]
thf_input = x_convLSTM_test_scaled[-1030:]

In [ ]:
res_lstm_train = model.predict(x_convLSTM_train_scaled[:1])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Calculate the vmin and vmax for both datasets
vmin = np.min(x_convLSTM_test_scaled[0])#, np.min(x_convLSTM_train_scaled[:1]))
vmax = np.max(x_convLSTM_test_scaled[0])#, np.max(x_convLSTM_train_scaled[:1]))

# Create subplots
fig, axs = plt.subplots(2, 3, figsize=(18, 12))

# Plotting the first row (res_lstm_train)
for i in range(3):
    im1 = axs[0, i].imshow(res_lstm_train[0, i].reshape((180, 360)), cmap='viridis', vmin=vmin, vmax=vmax)
    axs[0, i].set_title(f"res_lstm_train - Slice {i+1}")
    axs[0, i].axis('off')

# Plotting the second row (x_convLSTM_train_scaled)
for i in range(3):
    im2 = axs[1, i].imshow(x_convLSTM_test_scaled[:1][0, i].reshape((180, 360)), cmap='viridis', vmin=vmin, vmax=vmax)
    axs[1, i].set_title(f"x_convLSTM_train_scaled - Slice {i+1}")
    axs[1, i].axis('off')

# Add colorbars with the same range
cbar_ax1 = fig.add_axes([0.92, 0.6, 0.02, 0.3])  # Position for colorbar of the first row
cbar1 = fig.colorbar(im1, cax=cbar_ax1)
cbar_ax2 = fig.add_axes([0.92, 0.1, 0.02, 0.3])  # Position for colorbar of the second row
cbar2 = fig.colorbar(im2, cax=cbar_ax2)

plt.show()



In [ ]:
res_lstm_conv = res_lstm_restored[:,0,:,:,:]

true_state = y_convLSTM_test_restored[:,0,:,:,:]

In [ ]:
np.save('convlstm_4.npy',res_lstm_conv[0])

In [ ]:
def apply_mask(original_images, predicted_images):
    masked_predicted_images = []
    for original, predicted in zip(original_images, predicted_images):
        # 创建一个掩码，其中原始图像的像素值为0的位置为True
        mask = original == 0

        # 使用掩码更新预测图像，将对应于掩码为True的位置的像素值设置为0
        masked_predicted = np.where(mask, 0, predicted)
        masked_predicted_images.append(masked_predicted)

    return masked_predicted_images

In [ ]:
plt.imshow(res_lstm_conv[1000,:,:,:])

In [ ]:
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import time

# Initialize lists to store results
ssim_scores = []
psnr_scores = []
inference_times = []

# Define the number of cases
num_cases = 6

# Calculate the size of each case
case_size = len(res_lstm) // num_cases

# Loop over each case
for i in range(num_cases):
    # Get the subset of data for the current case
    res_subset = res_lstm_conv[i * case_size : (i + 1) * case_size]
    true_subset = true_state[i * case_size : (i + 1) * case_size]
    res_subset = apply_mask(true_subset, res_subset)
    # Initialize lists to store metrics for the current case
    case_ssim_scores = []
    case_psnr_scores = []
    case_inference_times = []

    # Calculate metrics for each sample in the current case
    for res, true in zip(res_subset, true_subset):
        # Measure inference time
        start_time = time.time()

        # Calculate SSIM
        ssim_score = ssim(res, true, multichannel=True)
        case_ssim_scores.append(ssim_score)

        # Calculate PSNR
        psnr_score = psnr(res, true, data_range=res.max() - res.min())
        case_psnr_scores.append(psnr_score)

        # Measure inference time
        end_time = time.time()
        inference_time = end_time - start_time
        case_inference_times.append(inference_time)

    # Calculate the average SSIM, PSNR, and inference time for the current case
    avg_ssim_score = sum(case_ssim_scores) / len(case_ssim_scores)
    avg_psnr_score = sum(case_psnr_scores) / len(case_psnr_scores)
    avg_inference_time = sum(case_inference_times) / len(case_inference_times)

    # Append the results to the lists
    ssim_scores.append(avg_ssim_score)
    psnr_scores.append(avg_psnr_score)
    inference_times.append(avg_inference_time)

# Print the results for each case
for i in range(num_cases):
    print(f"Case {i+1}:")
    print(f"Average SSIM: {ssim_scores[i]}")
    print(f"Average PSNR: {psnr_scores[i]}")
    print(f"Average Inference Time: {inference_times[i]} seconds")
    print()


# Add Sample

In [ ]:
TD_kriging = np.load('/content/2D_result_4-6.npy')[0]
tD_kriging = np.load('/content/3D_result_4-6.npy')[0]

TD_kriging_rotated = np.rot90(TD_kriging, k=-1)
tD_kriging_rotated = np.rot90(tD_kriging, k=-1)

vorcnn = np.load('/content/vorcnn.npy')
convlstm= np.load('/content/convlstm_4.npy')
ced = np.load('/content/ced_lstm_4.npy')

true_field = true_state[0]

In [ ]:
print("Shape of TD_kriging:", TD_kriging_rotated.shape)
print("Shape of tD_kriging:", tD_kriging_rotated.shape)
print("Shape of vorcnn:", vorcnn.shape)
print("Shape of convlstm:", convlstm.shape)
print("Shape of ced:", ced.shape)
print("Shape of true:", true_field.shape)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl

mpl.rcParams['font.size'] = 10
mpl.rcParams['axes.labelsize'] = 8

error_TD_kriging = true_field - TD_kriging_rotated.reshape((180,360,1))

# Calculate the errors for tD_kriging
error_tD_kriging = true_field - tD_kriging_rotated.reshape((180,360,1))


# Calculate the errors for vorcnn
error_vorcnn = true_field - vorcnn

# Calculate the errors for convlstm
error_convlstm = true_field - convlstm

# Calculate the errors for ced
error_ced = true_field - ced

abs_error_TD_kriging = np.abs(error_TD_kriging)

abs_error_tD_kriging = np.abs(error_tD_kriging)

abs_error_vorcnn = np.abs(error_vorcnn)

abs_error_convlstm = np.abs(error_convlstm)
abs_error_ced = np.abs(error_ced)


def plot_image_and_colorbar(ax, image, title, mask=None, cmap='viridis', cbar_labelsize=8, vmin=None, vmax=None):
    if mask is not None:
        image = np.ma.array(image, mask=mask)  # 使用掩码数组而不是直接修改数据
    im = ax.imshow(image, cmap=cmap, vmin=vmin, vmax=vmax)
    ax.set_title(title)
    ax.axis('off')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = plt.colorbar(im, cax=cax)
    cbar.ax.tick_params(labelsize=cbar_labelsize)  # 设置颜色条刻度的字体大小

# 假设true, TD_kriging_rotated等变量已经加载
mask = (y_test[0] == 0)  # 定义掩码

fig, axs = plt.subplots(4, 3, figsize=(18, 24), constrained_layout=True)
images = [true_field, TD_kriging_rotated, tD_kriging_rotated, vorcnn, convlstm, ced]
errors = [abs_error_TD_kriging, abs_error_tD_kriging, abs_error_vorcnn, abs_error_convlstm, abs_error_ced]
titles = ['True Field', '2D Kriging', '3D Kriging', '2D Kriging Error Map', '3D Kriging Error Map', 'Vor-CNN', 'Conv LSTM', 'CED-LSTM',
          'Vor-CNN Error Map', 'Conv LSTM Error Map', 'CED-LSTM Error Map']
cmaps = [cmap_viridis] * 6 + [cmap_hot] * 5

# 确定颜色范围
vmin, vmax = np.nanmin(true_field), np.nanmax(true_field)
error_vmin, error_vmax = 0, np.nanmax(abs_error_convlstm)

for i, ax in enumerate(axs.flat):
    if i < 6:
        plot_image_and_colorbar(ax, images[i], titles[i], mask=mask, cmap=cmaps[i], vmin=vmin, vmax=vmax)
    elif i < 11:
        plot_image_and_colorbar(ax, errors[i-6], titles[i], mask=mask, cmap=cmaps[i], vmin=error_vmin, vmax=error_vmax)
    else:
        ax.axis('off')

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

mpl.rcParams['font.size'] = 10
mpl.rcParams['axes.labelsize'] = 8

# Calculate errors
error_TD_kriging = true_field - TD_kriging_rotated.reshape((180,360,1))
error_tD_kriging = true_field - tD_kriging_rotated.reshape((180,360,1))
error_vorcnn = true_field - vorcnn
error_convlstm = true_field - convlstm
error_ced = true_field - ced

abs_error_TD_kriging = np.abs(error_TD_kriging)
abs_error_tD_kriging = np.abs(error_tD_kriging)
abs_error_vorcnn = np.abs(error_vorcnn)
abs_error_convlstm = np.abs(error_convlstm)
abs_error_ced = np.abs(error_ced)

mask = (y_test[0]==0)
# Plotting functions
def plot_image_and_colorbar(ax, image, title, cmap='viridis', vmin=None, vmax=None):
    if mask is not None:
        image = np.ma.array(image, mask=mask)
    im = ax.imshow(image, cmap=cmap, vmin=vmin, vmax=vmax)
    ax.set_title(title)
    ax.axis('off')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = plt.colorbar(im, cax=cax)
    cbar.ax.tick_params(labelsize=8)
    return cbar
# Create subplots
fig, axs = plt.subplots(4, 3, figsize=(18, 24), constrained_layout=True)

# True Field, 3D Kriging, 2D Kriging
cbar1 = plot_image_and_colorbar(axs[0, 0], true_field, 'True Field')
plot_image_and_colorbar(axs[0, 1], TD_kriging_rotated, '3D Kriging', vmin=cbar1.vmin, vmax=cbar1.vmax)
plot_image_and_colorbar(axs[0, 2], tD_kriging_rotated, '2D Kriging', vmin=cbar1.vmin, vmax=cbar1.vmax)

# Empty spaces
axs[1, 0].axis('off')

# Error maps for 3D Kriging and 2D Kriging
cbar2 = plot_image_and_colorbar(axs[1, 1], abs_error_TD_kriging, 'Error Map (3D Kriging)', vmin=cbar2.vmin, vmax=cbar2.vmax)
plot_image_and_colorbar(axs[1, 2], abs_error_tD_kriging, 'Error Map (2D Kriging)', vmin=cbar2.vmin, vmax=cbar2.vmax)

# Vor-CNN, ConvLSTM, CED-LSTM
plot_image_and_colorbar(axs[2, 0], vorcnn, 'Vor-CNN', vmin=cbar1.vmin, vmax=cbar1.vmax)
plot_image_and_colorbar(axs[2, 1], convlstm, 'ConvLSTM', vmin=cbar1.vmin, vmax=cbar1.vmax)
plot_image_and_colorbar(axs[2, 2], ced, 'CED-LSTM', vmin=cbar1.vmin, vmax=cbar1.vmax)

# Empty spaces
cbar3 = plot_image_and_colorbar(axs[3, 0], abs_error_vorcnn, 'Error Map (Vor-CNN)', vmin=cbar2.vmin, vmax=cbar2.vmax)

# Error maps for Vor-CNN, ConvLSTM, CED-LSTM
plot_image_and_colorbar(axs[3, 1], abs_error_convlstm, 'Error Map (ConvLSTM)', vmin=cbar2.vmin, vmax=cbar2.vmax)
cbar4 = plot_image_and_colorbar(axs[3, 2], abs_error_ced, 'Error Map (CED-LSTM)', vmin=cbar2.vmin, vmax=cbar2.vmax)

plt.show()



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

mpl.rcParams['font.size'] = 10
mpl.rcParams['axes.labelsize'] = 8

# Calculate errors
error_TD_kriging = true_field - TD_kriging_rotated.reshape((180,360,1))
error_tD_kriging = true_field - tD_kriging_rotated.reshape((180,360,1))
error_vorcnn = true_field - vorcnn
error_convlstm = true_field - convlstm
error_ced = true_field - ced

abs_error_TD_kriging = np.abs(error_TD_kriging)
abs_error_tD_kriging = np.abs(error_tD_kriging)
abs_error_vorcnn = np.abs(error_vorcnn)
abs_error_convlstm = np.abs(error_convlstm)
abs_error_ced = np.abs(error_ced)

mask = (y_test[0]==0)

# Plotting functions
def plot_image_and_colorbar(ax, image, title, cmap='viridis', vmin=None, vmax=None):
    if mask is not None:
        image = np.ma.array(image, mask=mask)
    im = ax.imshow(image, cmap=cmap, vmin=vmin, vmax=vmax)
    ax.set_title(title)
    ax.axis('off')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.08)
    cbar = plt.colorbar(im, cax=cax)
    cbar.ax.tick_params(labelsize=8)
    return cbar

# Create subplots
fig, axs = plt.subplots(4, 3, figsize=(28, 15))


vmin_error = np.min([np.min(abs_error_tD_kriging), np.min(abs_error_tD_kriging)])
vmax_error = np.max([np.max(abs_error_tD_kriging), np.max(abs_error_tD_kriging)])+1  # Subtract 3 for better visualization



# True Field, 3D Kriging, 2D Kriging
plot_image_and_colorbar(axs[0, 0], true_field, 'True Field', vmin=cbar1.vmin, vmax=cbar1.vmax)


plot_image_and_colorbar(axs[0, 1], TD_kriging_rotated, '3D Kriging', vmin=cbar1.vmin, vmax=cbar1.vmax)
plot_image_and_colorbar(axs[0, 2], tD_kriging_rotated, '2D Kriging', vmin=cbar1.vmin, vmax=cbar1.vmax)

# Empty spaces
axs[1, 0].axis('off')


plot_image_and_colorbar(axs[1, 1], abs_error_TD_kriging, 'Error Map (3D Kriging)', vmin=vmin_error, vmax=vmax_error)


plot_image_and_colorbar(axs[1, 2], abs_error_tD_kriging, 'Error Map (2D Kriging)', vmin=vmin_error, vmax=vmax_error)

# Vor-CNN, ConvLSTM, CED-LSTM
plot_image_and_colorbar(axs[2, 0], vorcnn, 'Vor-CNN', vmin=cbar1.vmin, vmax=cbar1.vmax)
plot_image_and_colorbar(axs[2, 1], convlstm, 'ConvLSTM', vmin=cbar1.vmin, vmax=cbar1.vmax)
plot_image_and_colorbar(axs[2, 2], ced, 'CED-LSTM', vmin=cbar1.vmin, vmax=cbar1.vmax)

# Empty spaces
cbar3 = plot_image_and_colorbar(axs[3, 0], abs_error_vorcnn, 'Error Map (Vor-CNN)', vmin=vmin_error, vmax=vmax_error)

# Error maps for Vor-CNN, ConvLSTM, CED-LSTM
plot_image_and_colorbar(axs[3, 1], abs_error_convlstm, 'Error Map (ConvLSTM)', vmin=vmin_error, vmax=vmax_error)
cbar4 = plot_image_and_colorbar(axs[3, 2], abs_error_ced, 'Error Map (CED-LSTM)', vmin=vmin_error, vmax=vmax_error)

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

mpl.rcParams['font.size'] = 12  # Increase font size
mpl.rcParams['axes.labelsize'] = 10  # Increase label font size

# Calculate errors
error_TD_kriging = true_field - TD_kriging_rotated.reshape((180,360,1))
error_tD_kriging = true_field - tD_kriging_rotated.reshape((180,360,1))
error_vorcnn = true_field - vorcnn
error_convlstm = true_field - convlstm
error_ced = true_field - ced

abs_error_TD_kriging = np.abs(error_TD_kriging)
abs_error_tD_kriging = np.abs(error_tD_kriging)
abs_error_vorcnn = np.abs(error_vorcnn)
abs_error_convlstm = np.abs(error_convlstm)
abs_error_ced = np.abs(error_ced)

mask = (y_test[0]==0)

# Plotting functions
def plot_image_and_colorbar(ax, image, title, cmap='viridis', vmin=None, vmax=None):
    if mask is not None:
        image = np.ma.array(image, mask=mask)
    im = ax.imshow(image, cmap=cmap, vmin=vmin, vmax=vmax)
    ax.set_title(title, fontsize=16)  # Increase title font size
    ax.axis('off')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.08)
    cbar = plt.colorbar(im, cax=cax)
    cbar.ax.tick_params(labelsize=14)
    return cbar

# Create subplots
fig, axs = plt.subplots(4, 3, figsize=(20, 13))


vmin_error = np.min([np.min(abs_error_tD_kriging), np.min(abs_error_tD_kriging)])
vmax_error = np.max([np.max(abs_error_tD_kriging), np.max(abs_error_tD_kriging)])+1  # Subtract 3 for better visualization



# True Field, 3D Kriging, 2D Kriging
plot_image_and_colorbar(axs[0, 0], true_field, 'True Field', vmin=cbar1.vmin, vmax=cbar1.vmax)


plot_image_and_colorbar(axs[0, 1], TD_kriging_rotated, '3D Kriging', vmin=cbar1.vmin, vmax=cbar1.vmax)
plot_image_and_colorbar(axs[0, 2], tD_kriging_rotated, '2D Kriging', vmin=cbar1.vmin, vmax=cbar1.vmax)

# Empty spaces
axs[1, 0].axis('off')


plot_image_and_colorbar(axs[1, 1], abs_error_TD_kriging, 'Error Map (3D Kriging)', vmin=vmin_error, vmax=vmax_error)


plot_image_and_colorbar(axs[1, 2], abs_error_tD_kriging, 'Error Map (2D Kriging)', vmin=vmin_error, vmax=vmax_error)

# Vor-CNN, ConvLSTM, CED-LSTM
plot_image_and_colorbar(axs[2, 0], vorcnn, 'Vor-CNN', vmin=cbar1.vmin, vmax=cbar1.vmax)
plot_image_and_colorbar(axs[2, 1], convlstm, 'ConvLSTM', vmin=cbar1.vmin, vmax=cbar1.vmax)
plot_image_and_colorbar(axs[2, 2], ced, 'CED-LSTM', vmin=cbar1.vmin, vmax=cbar1.vmax)

# Empty spaces
cbar3 = plot_image_and_colorbar(axs[3, 0], abs_error_vorcnn, 'Error Map (Vor-CNN)', vmin=vmin_error, vmax=vmax_error)

# Error maps for Vor-CNN, ConvLSTM, CED-LSTM
plot_image_and_colorbar(axs[3, 1], abs_error_convlstm, 'Error Map (ConvLSTM)', vmin=vmin_error, vmax=vmax_error)
cbar4 = plot_image_and_colorbar(axs[3, 2], abs_error_ced, 'Error Map (CED-LSTM)', vmin=vmin_error, vmax=vmax_error)

plt.show()